# Trabalhando com os Dados de Crítica de Jogos 

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy 
from sklearn import datasets, linear_model
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy as ps


C:\Users\dossantos.j.5\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Importando dados dos Games

In [2]:
Games = pd.read_csv('./data/Video_Games_Sales_as_at_22_Dec_2016.csv')

In [3]:
Games.head()


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


Fazer uma avaliação global sem considerar genero


In [5]:
Games.User_Score.describe()

count     10015
unique       96
top         tbd
freq       2425
Name: User_Score, dtype: object

In [6]:
#Removendo "tbd" de User_Score, para poder transformar em número
Games_filtered = Games[Games['User_Score'] != 'tbd']
#Games_filtered = Games_filtered[Games_filtered['Publisher']=='Nintendo']

In [8]:
#Importando train_test_split
from sklearn.model_selection import train_test_split

In [9]:
#Selecionando apenas as colunas que serão usadas, e removendo NaNs:
Games_Score_and_Sales = Games_filtered.loc[:,['Critic_Score','Global_Sales', 'User_Score', 'Year_of_Release', 'User_Count']].dropna(axis = 0)
#Convertendo User_Score para número:
Games_Score_and_Sales.User_Score = pd.to_numeric(Games_Score_and_Sales.User_Score)
#Aplicando Z-score para cada uma das medidas (subtrair a média e dividir pelo desvio padrão)
Games_Score_and_Sales.User_Score = (Games_Score_and_Sales.User_Score -Games_Score_and_Sales.User_Score.mean())/Games_Score_and_Sales.User_Score.std()
Games_Score_and_Sales.Critic_Score = (Games_Score_and_Sales.Critic_Score -Games_Score_and_Sales.Critic_Score.mean())/Games_Score_and_Sales.Critic_Score.std()
Games_Score_and_Sales.Year_of_Release = (Games_Score_and_Sales.Year_of_Release -Games_Score_and_Sales.Year_of_Release.mean())/Games_Score_and_Sales.Year_of_Release.std()
Games_Score_and_Sales.User_Count = (Games_Score_and_Sales.User_Count -Games_Score_and_Sales.User_Count.mean())/Games_Score_and_Sales.User_Count.std()


In [10]:
#Criando a regressão linear
regr = linear_model.LinearRegression()

"""Fazendo o split entre treino e test - 
aqui é possível selecionar quais as colunas serão usadas no modelo (no primeiro parâmetro) e qual o tamanho do teste"""

(Features_train, 
 Features_test, 
 Global_Sales_train, 
 Global_Sales_test) = train_test_split(Games_Score_and_Sales.loc[:,['Critic_Score', 'User_Count']], 
                                       Games_Score_and_Sales['Global_Sales'],
                                       test_size = 0.3
                                      )

#Features só precisam de reshape caso tenham apenas 1 coluna:
#Features_train = Features_train.values.reshape(-1,1)
#Features_test = Features_test.values.reshape(-1,1)

#Reshape de Global_Sales treino e teste
Global_Sales_train = Global_Sales_train.values.reshape(-1,1)
Global_Sales_test = Global_Sales_test.values.reshape(-1,1)

In [11]:
#Treinando a regressão
regr.fit(Features_train, Global_Sales_train)
print(regr.coef_)
print(regr.intercept_)

[[0.36849389 0.51583421]]
[0.80998355]


In [13]:
print("R2 da amostra de treino: " + str(regr.score(Features_train, Global_Sales_train)))
print("R2 da amostra de teste: " + str(regr.score(Features_test, Global_Sales_test)))

R2 da amostra de treino: 0.10827754305417069
R2 da amostra de teste: 0.04241081178148998


Filtrando Jogos sem avaliação da crítica

In [5]:
Games2=Games[pd.notnull(Games['Critic_Score'])]
Games2.head(10)


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.44,6.94,4.70,2.24,28.32,87.0,80.0,8.4,594.0,Nintendo,E
11,Mario Kart DS,DS,2005.0,Racing,Nintendo,9.71,7.47,4.13,1.90,23.21,91.0,64.0,8.6,464.0,Nintendo,E
13,Wii Fit,Wii,2007.0,Sports,Nintendo,8.92,8.03,3.60,2.15,22.70,80.0,63.0,7.7,146.0,Nintendo,E
14,Kinect Adventures!,X360,2010.0,Misc,Microsoft Game Studios,15.00,4.89,0.24,1.69,21.81,61.0,45.0,6.3,106.0,Good Science Studio,E
15,Wii Fit Plus,Wii,2009.0,Sports,Nintendo,9.01,8.49,2.53,1.77,21.79,80.0,33.0,7.4,52.0,Nintendo,E


Filtrando jogos sem dados de vendas (se existirem)

In [6]:
Games3=Games2[pd.notnull(Games2['Global_Sales'])]
Games3.head(10)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.44,6.94,4.70,2.24,28.32,87.0,80.0,8.4,594.0,Nintendo,E
11,Mario Kart DS,DS,2005.0,Racing,Nintendo,9.71,7.47,4.13,1.90,23.21,91.0,64.0,8.6,464.0,Nintendo,E
13,Wii Fit,Wii,2007.0,Sports,Nintendo,8.92,8.03,3.60,2.15,22.70,80.0,63.0,7.7,146.0,Nintendo,E
14,Kinect Adventures!,X360,2010.0,Misc,Microsoft Game Studios,15.00,4.89,0.24,1.69,21.81,61.0,45.0,6.3,106.0,Good Science Studio,E
15,Wii Fit Plus,Wii,2009.0,Sports,Nintendo,9.01,8.49,2.53,1.77,21.79,80.0,33.0,7.4,52.0,Nintendo,E


In [7]:
X=Games3['Global_Sales']
Y=Games3['Critic_Count']
NP_X=np.array(X)
NP_Y=np.array(Y)
print(NP_X)
print(NP_Y)
corr = np.corrcoef(NP_X[:],NP_Y[:])
print("Correlation: " + str(corr))


[8.253e+01 3.552e+01 3.277e+01 ... 1.000e-02 1.000e-02 1.000e-02]
[51. 73. 73. ... 10. 12.  6.]
Correlation: [[1.         0.30357083]
 [0.30357083 1.        ]]


Rank Correlation - Spearman

In [ ]:
X_rank=np.array(scipy.stats.stats.rankdata(X))
print(X_rank)

Y_rank=np.array(scipy.stats.stats.rankdata(Y))
print(Y_rank)

corr = np.corrcoef(X_rank[:],Y_rank[:])
print("Spearman Correlation: " + str(corr))

Será que filtrando para as vendas no japão isto acontece?

In [ ]:
Games4=Games3[pd.notnull(Games3['JP_Sales'])]
Games4.head(500)

In [ ]:
X=Games4['JP_Sales']
Y=Games4['Critic_Count']
NP_X=np.array(X)
NP_Y=np.array(Y)
print(NP_X)
print(NP_Y)
corr = np.corrcoef(NP_X[:],NP_Y[:])
print("Correlation: " + str(corr))
X_rank=np.array(scipy.stats.stats.rankdata(X))
print(X_rank)

Y_rank=np.array(scipy.stats.stats.rankdata(Y))
print(Y_rank)

corr = np.corrcoef(X_rank[:],Y_rank[:])
print("Spearman Correlation: " + str(corr))

Será que isto muda por genero? Selecionarei RPG's

In [ ]:
Games5=Games3[Games3['Genre']=="Role-Playing"]
Games5.head(500)

In [ ]:
X=Games5['Global_Sales']
Y=Games5['Critic_Count']
NP_X=np.array(X)
NP_Y=np.array(Y)
corr = np.corrcoef(NP_X[:],NP_Y[:])
print("Correlation: " + str(corr))

X_rank=np.array(scipy.stats.stats.rankdata(X))
Y_rank=np.array(scipy.stats.stats.rankdata(Y))

corr = np.corrcoef(X_rank[:],Y_rank[:])
print("Spearman Correlation: " + str(corr))

Com RPG's mudou o resultado será que não vale a pena filtrar dados mais recentes?

In [ ]:
Games6=Games5[Games5['Year_of_Release'] > 2008]
Games6.head(500)

In [ ]:
X=Games6['Global_Sales']
Y=Games6['Critic_Count']
NP_X=np.array(X)
NP_Y=np.array(Y)
corr = np.corrcoef(NP_X[:],NP_Y[:])
print("Correlation: " + str(corr))

X_rank=np.array(scipy.stats.stats.rankdata(X))
Y_rank=np.array(scipy.stats.stats.rankdata(Y))

corr = np.corrcoef(X_rank[:],Y_rank[:])
print("Spearman Correlation: " + str(corr))

Chegamos a uma correlação boa, vamos ajustar uma Regressão Linear

In [ ]:
# Create linear regression object
regr = linear_model.LinearRegression()
X_train=Y.reshape(len(Y),1)
Y_train=X.reshape(len(X),1)

# Train the model using the training sets
regr.fit(X_train, Y_train)

beta1=regr.coef_

intercepto=regr.intercept_

print(beta1)
print(intercepto)

#R quadrado 
regr.score(X_train,Y_train)

Eai, quais conclusões chegaram????

Utilizando o statmodels

In [ ]:
Games5.head()

In [ ]:
def troca(x):
    try:
        return float(x)
    except ValueError as err:
        return np.nan

Games5["User_Score"]=Games5["User_Score"].apply( troca ) 



In [ ]:
y,X = ps.dmatrices('Global_Sales ~ Critic_Score + User_Score +Publisher',data=Games5, return_type='dataframe')
print(type(y))
Games5.head(500)


In [ ]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# For each X, calculate VIF and save in dataframe
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif.round(1)

In [ ]:
Games.get_values()

Analise do Heatmap - mapa de calor

In [ ]:
corr = Games6.corr()

In [ ]:
import seaborn as sns
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)